In [1]:
import boto3
from aws_mturk_key import *

In [85]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(open("my_question.xml"))

In [2]:
region_name = 'us-east-1'

endpoint_url = 'https://mturk-requester-sandbox.us-east-1.amazonaws.com'

# Uncomment this line to use in production
# endpoint_url = 'https://mturk-requester.us-east-1.amazonaws.com'

client = boto3.client(
    'mturk',
    endpoint_url=endpoint_url,
    region_name=region_name,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
)

# This will return $10,000.00 in the MTurk Developer Sandbox
print(client.get_account_balance()['AvailableBalance'])

10000.00


# Create Hit Sample

In [42]:
import sys
import boto3

create_hits_in_live = False

environments = {
        "live": {
            "endpoint": "https://mturk-requester.us-east-1.amazonaws.com",
            "preview": "https://www.mturk.com/mturk/preview",
            "manage": "https://requester.mturk.com/mturk/manageHITs",
            "reward": "0.00"
        },
        "sandbox": {
            "endpoint": "https://mturk-requester-sandbox.us-east-1.amazonaws.com",
            "preview": "https://workersandbox.mturk.com/mturk/preview",
            "manage": "https://requestersandbox.mturk.com/mturk/manageHITs",
            "reward": "0.1"
        },
}
mturk_environment = environments["live"] if create_hits_in_live else environments["sandbox"]

In [104]:
# use profile if one was passed as an arg, otherwise
# profile_name = sys.argv[1] if len(sys.argv) >= 2 else None
# session = boto3.Session(profile_name=profile_name)
client = boto3.client(
    service_name='mturk',
    region_name='us-east-1',
    endpoint_url=mturk_environment['endpoint'],
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
)

# Test that you can connect to the API by checking your account balance
user_balance = client.get_account_balance()

# In Sandbox this always returns $10,000. In live, it will be your acutal balance.
print("Your account balance is {}".format(user_balance['AvailableBalance']))

# The question we ask the workers is contained in this file.
question_sample = open("my_question.xml", "r").read().format("".join(open("question.html", "r").readlines()))

# Example of using qualification to restrict responses to Workers who have had
# at least 80% of their assignments approved. See:
# http://docs.aws.amazon.com/AWSMechTurk/latest/AWSMturkAPI/ApiReference_QualificationRequirementDataStructureArticle.html#ApiReference_QualificationType-IDs
worker_requirements = [{
    'QualificationTypeId': '000000000000000000L0',
    'Comparator': 'GreaterThanOrEqualTo',
    'IntegerValues': [80],
    'RequiredToPreview': True,
}]

# Create the HIT
response = client.create_hit(
    MaxAssignments=10,
    LifetimeInSeconds=28*24*60*60,
    AssignmentDurationInSeconds=2*24*60*60,
    Reward=mturk_environment['reward'],
    Title='The Memory Game (~5min, bonus up to $2.5)',
    Keywords='memorability, image, memory',
    Description='An image-based memorability test.',
    Question=question_sample,
    QualificationRequirements=worker_requirements,
)

# The response included several fields that will be helpful later
hit_type_id = response['HIT']['HITTypeId']
hit_id = response['HIT']['HITId']
print("\nCreated HIT: {}".format(hit_id))

print("\nYou can work the HIT here:")
print(mturk_environment['preview'] + "?groupId={}".format(hit_type_id))

print ("\nAnd see results here:")
print (mturk_environment['manage'])

Your account balance is 10000.00

Created HIT: 336OE47KI2Y5ZHMJ1UTQUBCIZJMWV0

You can work the HIT here:
https://workersandbox.mturk.com/mturk/preview?groupId=3U729HRE5ASCILF2WV495PFWKOTMEA

And see results here:
https://requestersandbox.mturk.com/mturk/manageHITs


# Retrieve And Approve Hit Sample

In [105]:
import sys
import boto3
from xml.dom.minidom import parseString

# hit_id = "3JTPR5MTZSQX6NDN6PSA22OOZOB5K6"

# By default, we use the free-to-use Sandbox
create_hits_in_live = False

environments = {
        "live": {
            "endpoint": "https://mturk-requester.us-east-1.amazonaws.com",
            "preview": "https://www.mturk.com/mturk/preview",
            "manage": "https://requester.mturk.com/mturk/manageHITs",
            "reward": "0.00"
        },
        "sandbox": {
            "endpoint": "https://mturk-requester-sandbox.us-east-1.amazonaws.com",
            "preview": "https://workersandbox.mturk.com/mturk/preview",
            "manage": "https://requestersandbox.mturk.com/mturk/manageHITs",
            "reward": "0.1"
        },
}
mturk_environment = environments["live"] if create_hits_in_live else environments["sandbox"]

client = boto3.client(
    service_name='mturk',
    region_name='us-east-1',
    endpoint_url=mturk_environment['endpoint'],
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
)

In [106]:
hit = client.get_hit(HITId=hit_id)
print ('Hit {} status: {}'.format(hit_id, hit['HIT']['HITStatus']))
response = client.list_assignments_for_hit(
    HITId=hit_id,
    AssignmentStatuses=['Submitted', 'Approved'],
    MaxResults=10,
)

def get_answer_of_xml(answer_xml):
    ans_dict = {}
    for answer in answer_xml.getElementsByTagName('Answer'):
        quest = answer.getElementsByTagName("QuestionIdentifier")[0].childNodes[0].nodeValue
        ans = answer.getElementsByTagName("FreeText")[0].childNodes[0].nodeValue
        ans_dict[quest] = ans
    return ans_dict

assignments = response['Assignments']
print ('The number of submitted assignments is {}'.format(len(assignments)))
for assignment in assignments:
    worker_id = assignment['WorkerId']
    assignment_id = assignment['AssignmentId']
    answer_xml = parseString(assignment['Answer'])

    # the answer is an xml document. we pull out the value of the first
    # //QuestionFormAnswers/Answer/FreeText
    answer = get_answer_of_xml(answer_xml)

#     print('The Worker with ID {} submitted assignment {} and gave the answer "{}"'.format(worker_id, assignment_id, only_answer))
    print('[{}] The Worker with ID {} gave the answer "{}"'.format(assignment['AssignmentStatus'], worker_id, answer))

    # Approve the Assignment (if it hasn't already been approved)
#     if assignment['AssignmentStatus'] == 'Submitted':
#         print ('Approving Assignment {}'.format(assignment_id))
#         client.approve_assignment(
#             AssignmentId=assignment_id,
#             RequesterFeedback='good',
#             OverrideRejection=False,
#         )
    client.reject_assignment(
        AssignmentId=assignment_id,
        RequesterFeedback='Too much invalid games',
    )

Hit 336OE47KI2Y5ZHMJ1UTQUBCIZJMWV0 status: Assignable
The number of submitted assignments is 1
[Submitted] The Worker with ID A1J0G097HN1OUA gave the answer "{'nickname': 'rdfg', 'password': 'asdas'}"
